In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [5]:
from parser.MAV.models import DeclarativeBase

DeclarativeBase.metadata.create_all(engine)

2022-11-09 17:13:15,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:13:15,055 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("RSS_events_MAV")
2022-11-09 17:13:15,057 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:13:15,060 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("RAW_events_MAV")
2022-11-09 17:13:15,062 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:13:15,063 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
from feed_handler.feed_consumer import FeedConsumer
from datetime import datetime
from time import mktime

Feed = FeedConsumer()
d = Feed.fetch()
time_struct = d.entries[0]['published_parsed']
date = datetime.fromtimestamp(mktime(time_struct))
print(date)

2022-11-09 16:10:31


In [4]:
from sqlalchemy.orm import Session
from parser.MAV.feed_parser import FeedParser

with Session(engine) as session:
    entry = FeedParser.parse_entry(d.entries[0])
    entry_dump = FeedParser.dump_entry(d.entries[0],entry.uuid)
    session.add_all([entry,entry_dump])
    session.commit()

2022-11-09 17:12:44,866 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:12:44,872 INFO sqlalchemy.engine.Engine INSERT INTO "RSS_events_MAV" (uuid, id, last_modification, update_uuid, title) VALUES (?, ?, ?, ?, ?)
2022-11-09 17:12:44,874 INFO sqlalchemy.engine.Engine [generated in 0.00237s] (<memory at 0x0000013ED73D9900>, '113333', '2022-11-09', None, 'Kimarad két délutáni S210-es vonat')
2022-11-09 17:12:44,878 INFO sqlalchemy.engine.Engine INSERT INTO "RAW_events_MAV" (event_uuid, event_dict) VALUES (?, ?)
2022-11-09 17:12:44,880 INFO sqlalchemy.engine.Engine [generated in 0.00164s] (None, '{"title": "Kimarad k\\u00e9t d\\u00e9lut\\u00e1ni S210-es vonat", "title_detail": {"type": "text/plain", "language": "hu", "base": "https://www.mavcs ... (568 characters truncated) ... : "M\\u00c1V.hu szerkeszt\\u0151s\\u00e9g", "author_detail": {"name": "M\\u00c1V.hu szerkeszt\\u0151s\\u00e9g"}, "id": "113333", "guidislink": false}')
2022-11-09 17:12:44,882 INFO sqlalchemy.engine.En

In [8]:
from sqlalchemy import select
from parser.MAV.models import MAVEvent
with Session(engine) as session:
    events = session.query(MAVEvent)
    print(events.first().id)


2022-11-09 17:18:58,809 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:18:58,812 INFO sqlalchemy.engine.Engine SELECT "RSS_events_MAV".uuid AS "RSS_events_MAV_uuid", "RSS_events_MAV".id AS "RSS_events_MAV_id", "RSS_events_MAV".last_modification AS "RSS_events_MAV_last_modification", "RSS_events_MAV".update_uuid AS "RSS_events_MAV_update_uuid", "RSS_events_MAV".title AS "RSS_events_MAV_title" 
FROM "RSS_events_MAV"
 LIMIT ? OFFSET ?
2022-11-09 17:18:58,813 INFO sqlalchemy.engine.Engine [cached since 13.8s ago] (1, 0)
113333
2022-11-09 17:18:58,820 INFO sqlalchemy.engine.Engine ROLLBACK
